In [1]:
import logging 
import random 
from copy import copy
import platform 
from collections import Counter 
from gx_utils import *
from math import exp
import time
import itertools
import numpy as np

In [2]:
def problem(N, seed = None): 
    random.seed(seed) 
    return [ list(set(random.randint(0,N-1) for n in  range(random.randint(N //5 ,N//2))))
                 for n in range(random.randint(N,N*5))
    ]

In [9]:
def bypassbranch(subset, i):#bypass a branch 
    for j, sub in reversed( list(enumerate(subset[0:i-1]))) :
    
        if sub == 0:
            subset[j] = 1
            return subset, j+1

    return subset, 0

def nextvertex(subset, i, m):
    if i < m:
        subset[i] = 0
        return subset, i+1
    else:
        for j, sub in reversed( list(enumerate(subset[0:m-1]))) : 
            if sub == 0:
                subset[j] = 1
                return subset, j+1
                
    return subset, 0

In [28]:
def BB(N):
    goal =  set(range(N))
    sets = problem(N,seed =42) 
    costs = list( len(s) for s  in sets)
    subset = [1 for x in range(len(sets))]#all sets in
    subset[0] = 0
    bestCost = sum(costs) #actually the worst cost 
    i = 1
    Nodes = list()
    Nodes.append(subset)
    z=time.time()
    solution = list()
    while i > 0:

        if i < len(sets):
            cost, temp_Set = 0, set()# temp for temporary
            for k , (sub , c) in enumerate(zip(subset[0:i],costs[0:i])):
                cost += sub*c   #if 1 adds the cost to total
                if sub == 1: temp_Set.update(set(sets[k]))#if 1 add the set to the cover
            #cost =  sum(np.multiply(subset,costs))
                
            if cost > bestCost:#if the cost is larger than the currently best one, we can bypass the branch 
                subset, i = bypassbranch(subset, i)
                continue
            temp_Set.update(set(itertools.chain(*sets[i::])))
            if temp_Set != goal:#that means that the set was essential at this point to complete the uni.
                subset, i = bypassbranch(subset, i)
            else:
                subset, i = nextvertex(subset, i, len(sets))
            Nodes.append(subset)
        
        else: 
            
            cost , final_Set = 0, set() 
            for k , (sub , c) in enumerate(zip(subset,costs)):
                cost+= sub*c
                if sub == 1 :final_Set.update(set(sets[k]))
            if cost < bestCost and final_Set == goal: 
                bestCost = cost 
                bestSubset = subset[:]
                print(subset)
                print(bestCost)
            subset , i = nextvertex(subset , i , len(sets))

        if (time.time()-z > 600.0):
            i = 0
    for x, sub in enumerate(bestSubset):
        if sub==1:
            solution.append(sets[x])
    unique_data = [list(i) for i  in set(tuple(i) for i in Nodes)]

    logging.info(
        f"Branch and Bound solution for N={N}: Cost ={bestCost}  / visited Nodes={len(unique_data)} / Search time= {time.time()-z} (bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%)"
    )
    logging.debug(f"{solution}")
    

In [29]:
logging.getLogger().setLevel(logging.INFO)
for N in [ 5]:
    BB(N)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0]
6
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0]
5


INFO:root:Branch and Bound solution for N=5: Cost =5  / visited Nodes=1 / Search time= 0.48158788681030273 (bloat=0%)


SIMULATED ANNEALING + GREEDY


In [2]:
def  greedy_2(N,subsets):
    goal = set(range(N))
    
    cost=0
    
    elements = set(e for s in subsets for e in s)
    
    covered = set()
    solution = []
    while covered != elements:
        subset = max(subsets, key=lambda s: len(set(s) - covered)/len(s))
        solution.append(subset)
        
        covered |= set(subset)
    return solution

In [3]:




def sa_algorithm(state,all_lists, temp_start = 350, update_temp = lambda t : 0.9999 * t, stop_criterion = lambda t : t <= 0.01):
    state = set (tuple(_) for _ in state)
    all_lists = set (tuple(_) for _ in all_lists)
    GOAL =  set(range(N))
    
    def tweak(solution): 
        new_solution = set(solution)
        while new_solution and random.random() < 0.7: 
            r = random.choice(list(new_solution))
            new_solution.remove(r)
        while random.random() < 0.7:
            a = random.choice(list(all_lists-solution))
            new_solution.add(a)
        return new_solution       
   
   
   
    def evaluate(state):
            cnt = Counter()
            cnt.update( sum((e for e in state), start = () ))
            return len(cnt), -cnt.total()
        
    
    temp = temp_start
    best_solution = copy(state)
    curr_solution = copy(state)
    # print("Inside sa: ", instance.get_total_distance_and_vehicles())
    while not stop_criterion(temp):
        # print(temp, incumb_vhcls, incumb_dist)
        
        new_solution = tweak(curr_solution)
        current_eval= evaluate(curr_solution)
        new_eval = evaluate(new_solution)


        

        if new_eval > current_eval \
            or random.random() < exp(- (abs(  sum(current_eval)-sum(new_eval))) / temp):
            curr_solution = new_solution

            if evaluate(curr_solution) > evaluate(best_solution) :
                best_solution = curr_solution
                print(sum(len(_) for _ in best_solution))
                logging.debug(f"New solution: {evaluate(best_solution)}")
        temp = update_temp(temp)
    return best_solution
                


In [6]:
logging.getLogger().setLevel(logging.INFO)

for N in [ 100, 500, 1000]:
    all_lists = problem(N, seed =42) 
      
    init_state = greedy_2(N,all_lists)
    print("GREEDY 2 COST"+str(sum(len(_) for _ in init_state))) 
    solution = sa_algorithm(init_state,all_lists)
    logging.info(
        f" Solution for N={N:,}: "
        + f"w={sum(len(_) for _ in solution):,} "
        + f"(bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%)"
    )

GREEDY 2 COST171


INFO:root: Solution for N=100: w=171 (bloat=71%)


GREEDY 2 COST1256


INFO:root: Solution for N=500: w=1,256 (bloat=151%)


GREEDY 2 COST2913


KeyboardInterrupt: 